In [1]:
!pip install spectral
!pip install keras-flops
from keras_flops import get_flops
import tensorflow as tf
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization,concatenate
from keras.layers import Dropout, Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv
from tensorflow.keras.utils import plot_model

from plotly.offline import init_notebook_mode

import time
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral


#from tensorflow.keras.utils import np_utils
init_notebook_mode(connected=True)
%matplotlib inline

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

#config = ConfigProto()
#config.gpu_options.allow_growth = True
#session = InteractiveSession(config=config)
## Mounting Colab
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

     |████████████████████████████████| 212 kB 5.2 MB/s 


Mounted at /content/drive


In [2]:
def loadData(name):
    data_path = os.path.join(os.getcwd(),'/content/drive/My Drive/hyperspectralpaper/Data')
    if name == 'IP':
        #data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/indianpca30.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/indianpca15.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/indianpca20.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/indianpca25.npy')

        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        #data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/sapca30.npy')
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        #data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/PUpca30.npy')
        data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/PUpca15.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/PUpca20.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/PUpca25.npy')

        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
        
    elif name == 'KSC':
        #data = sio.loadmat(os.path.join(data_path, 'Pavia.mat'))['pavia']
        data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/kscpca30.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/kscpca15.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/kscpca20.npy')
        # data = np.load('/content/drive/My Drive/hyperspectralpaper/Data/kscpca25.npy')

        labels = sio.loadmat(os.path.join(data_path, 'KSC_gt.mat'))['KSC_gt']
    #elif name == 'BO':
        #data = sio.loadmat(os.path.join(data_path, 'Botswana.mat'))['Botswana']
        #labels = sio.loadmat(os.path.join(data_path, 'Botswana_gt.mat'))['Botswana_gt']
    return data, labels

In [3]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [140]:
# GLOBAL VARIABLES
dataset = 'IP'
test_ratio = 0.90
batchsize = 64
epoch = 300
K = 30
windowSize = 15

In [141]:
X, y = loadData(dataset)
print(X.shape)
print(y.shape)
X, y = createImageCubes(X, y, windowSize=windowSize)
print(X.shape)
print(y.shape)

(145, 145, 30)
(145, 145)
(10249, 15, 15, 30)
(10249,)


In [142]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)
print(Xtrain.shape)
print(Xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(1024, 15, 15, 30)
(9225, 15, 15, 30)
(1024,)
(9225,)


In [143]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, K,1)
print(Xtrain.shape)
ytrain = np_utils.to_categorical(ytrain)
print(ytrain.shape)

(1024, 15, 15, 30, 1)
(1024, 16)


In [144]:
S = windowSize
L = K
if dataset == 'PU':
    output_units = 9
elif dataset == 'IP' or dataset == 'SA':
     output_units = 16
elif dataset == 'KSC':
     output_units = 13

In [145]:
## input layer
input_layer = Input((S, S, L, 1))

## convolutional layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
BN_layer1=BatchNormalization(momentum=0.95, epsilon=1e-5)(conv_layer1)
conv_layer2 = Conv3D(filters=8, kernel_size=(3, 3, 5), activation='relu')(BN_layer1)
BN_layer2=BatchNormalization(momentum=0.95, epsilon=1e-5)(conv_layer2)
conv_layer3 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(BN_layer2)
BN_layer3=BatchNormalization(momentum=0.95, epsilon=1e-5)(conv_layer3)

conv_layer4 = Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same',activation='relu')(BN_layer3)
BN_layer4=BatchNormalization(momentum=0.95, epsilon=1e-5)(conv_layer4)
ADD_layer1=concatenate([BN_layer3,BN_layer4],axis=4)

conv_layer5 = Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same',activation='relu')(ADD_layer1)
BN_layer5=BatchNormalization(momentum=0.95, epsilon=1e-5)(conv_layer5)
ADD_layer2=concatenate([ADD_layer1,BN_layer5],axis=4)

conv_layer6 = Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same',activation='relu')(ADD_layer2)
BN_layer6=BatchNormalization(momentum=0.95, epsilon=1e-5)(conv_layer6)
ADD_layer3=concatenate([BN_layer6,ADD_layer2],axis=4)

# print(ADD_layer3._keras_shape)
# conv3d_shape = ADD_layer3._keras_shape

conv_layer7 = Conv3D(filters=16, kernel_size=(1, 1, 1),activation='relu')(ADD_layer3)
BN_layer7=BatchNormalization(momentum=0.95, epsilon=1e-5)(conv_layer7)
Pool_layer1=keras.layers.AveragePooling3D(pool_size=(3,3,3))(BN_layer7)

flatten_layer = Flatten()(Pool_layer1)

# fully connected layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)


In [146]:
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 15, 15, 30,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv3d_42 (Conv3D)             (None, 13, 13, 24,   512         ['input_7[0][0]']                
                                8)                                                                
                                                                                                  
 batch_normalization_42 (BatchN  (None, 13, 13, 24,   32         ['conv3d_42[0][0]']              
 ormalization)                  8)                                                          

In [147]:
flops = get_flops(model, batch_size=1)
#print(f"FLOPS: {flops / 10 ** 9:.03} G")
print(f"FLOPS: {flops}")

FLOPS: 117361856


In [148]:
# plot_model(model, to_file='3DDWTDENSEBO.png',show_shapes=True)

In [149]:
# compiling the model
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



In [150]:
# checkpoint
# filepath = "/content/drive/My Drive/hyperspectralpaper/model/DWTDENSEBO-model.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

filepath = "/content/drive/My Drive/hyperspectralpaper/model/DWTDENSEBO-model.hdf5"
my_callbacks=[
  EarlyStopping(monitor='accuracy', patience=30),
  ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
]


In [ ]:
start=time.time()
# history = model.fit(x=Xtrain, y=ytrain, batch_size=batchsize, epochs=epoch,callbacks=callbacks_list)
history = model.fit(x=Xtrain, y=ytrain, batch_size=batchsize, epochs=epoch,callbacks=my_callbacks)
end=time.time()
print(end-start)

Epoch 1/300
16/16 [==============================] - ETA: 0s - loss: 2.2175 - accuracy: 0.3125
Epoch 00001: accuracy improved from -inf to 0.31250, saving model to /content/drive/My Drive/hyperspectralpaper/model/DWTDENSEBO-model.hdf5
16/16 [==============================] - 2s 36ms/step - loss: 2.2175 - accuracy: 0.3125
Epoch 2/300
15/16 [===========================>..] - ETA: 0s - loss: 1.2320 - accuracy: 0.6146
Epoch 00002: accuracy improved from 0.31250 to 0.61914, saving model to /content/drive/My Drive/hyperspectralpaper/model/DWTDENSEBO-model.hdf5
16/16 [==============================] - 1s 35ms/step - loss: 1.2134 - accuracy: 0.6191
Epoch 3/300
16/16 [==============================] - ETA: 0s - loss: 0.7326 - accuracy: 0.7666
Epoch 00003: accuracy improved from 0.61914 to 0.76660, saving model to /content/drive/My Drive/hyperspectralpaper/model/DWTDENSEBO-model.hdf5
16/16 [==============================] - 1s 35ms/step - loss: 0.7326 - accuracy: 0.7666
Epoch 4/300
16/16 [======

In [ ]:
# plt.figure(figsize=(5,5))
# plt.grid()
# plt.plot(history.history['loss'])
# #plt.plot(history.history['val_loss'])
# plt.ylabel('Loss')
# plt.xlabel('Epochs')
# plt.legend(['Training','Validation'], loc='upper right')
# #plt.savefig("loss_curve.pdf")
# plt.show()
# plt.figure(figsize=(5,5))
# plt.ylim(0,1.1)
# plt.grid()
# plt.plot(history.history['accuracy'])
# #plt.plot(history.history['val_acc'])
# plt.ylabel('Accuracy')
# plt.xlabel('Epochs')
# plt.legend(['Training','Validation'])
# #plt.savefig("acc_curve.pdf")
# plt.show()

In [ ]:
# load best weights
from tensorflow.keras.models import load_model
#model=keras.models.load_model("best-model.hdf5")
model=load_model('/content/drive/My Drive/hyperspectralpaper/model/DWTDENSEBO-model.hdf5')
Xtest = Xtest.reshape(-1, windowSize, windowSize, K,1)
print(Xtest.shape)
ytest = np_utils.to_categorical(ytest)
print(ytest.shape)
# Y_pred_test = model.predict(Xtest)
# y_pred_test = np.argmax(Y_pred_test, axis=1)
# classification = classification_report(np.argmax(ytest, axis=1), y_pred_test,digits=6)
# print(classification)

In [ ]:
def reports (X_test,y_test,name):
    start = time.time()
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    end = time.time()
    print(end - start)
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']
    elif name == 'KSC':
        target_names = ['Water','Mud_flats','Salt_marsh','Catiail_marsh','Spartina_marsh','Graminoid_marsh',
                       'Hardwood_swamp','Oak-Broadleaf','Slash_Pine','CP-Oak','CP-hammock','Willow swamp',
                       'Scrub']
    
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names,digits=6)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=batchsize)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100


In [ ]:
classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(Xtest,ytest,dataset)

In [ ]:
classification
print(classification)
print(Test_loss)
print(Test_accuracy)
print(aa)
print(oa)
print(kappa)

In [ ]:
# def Patch(data,height_index,width_index):
#     height_slice = slice(height_index, height_index+PATCH_SIZE)
#     width_slice = slice(width_index, width_index+PATCH_SIZE)
#     patch = data[height_slice, width_slice, :]
    
#     return patch


In [ ]:
# load the original image
# X, y = loadData(dataset)

In [ ]:
# height = y.shape[0]
# width = y.shape[1]
# PATCH_SIZE = windowSize
# numComponents = K

In [ ]:
# X = padWithZeros(X, PATCH_SIZE//2)

In [ ]:
# calculate the predicted image
# outputs = np.zeros((height,width))
# for i in range(height):
#     for j in range(width):
#         target = int(y[i,j])
#         if target == 0 :
#             continue
#         else :
#             image_patch=Patch(X,i,j)
#             X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1], image_patch.shape[2], 1).astype('float32')                                   
#             prediction = (model.predict(X_test_image))
#             prediction = np.argmax(prediction, axis=1)
#             outputs[i][j] = prediction+1

In [ ]:
# from spectral import *
# from scipy.io import loadmat
# import matplotlib.pyplot as plt
# img, la = loadData(dataset)

# ground_truth = spectral.imshow(classes = y,figsize =(7,7))
# gt_map = np.zeros(shape=(height,width,3))
# groundtruth
# indianpines_colors = np.array([#[255,255,255],
#                 [0,0,0],
#                 [255,254,137],[3,28,241],[255,89,1],[5,255,133],               
#                 [255,2,251],[89,1,255],[3,171,255],[12,255,7],
#                 [172,175,84],[160,78,158],[101,173,255],[60,91,112],
#                 [104,192,63],[139,69,46],[119,255,172],[254,255,3]])
# #print(indianpines_colors[la[3,3]])
# gt_map = np.zeros(shape=(height,width,3))
# for m in range(height):
#   for n in range(width):
#     gt_map[m,n,:] = indianpines_colors[la[m,n]]


# # print(gt_map[3,3,:])

# # fig = plt.figure(figsize=(7,7))
# plt.figure("groundtruth")
# plt.imshow(gt_map.astype('uint8'))
# plt.show()

In [ ]:
# predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))
# predict_image

# print(outputs.astype(int)[3,3])

# pr_map = np.zeros(shape=(height,width,3))
# for h in range(height):
#   for w in range(width):
#     pr_map[h,w,:] = indianpines_colors[outputs.astype(int)[h,w]]

# plt.figure("dwtdense")
# plt.imshow(pr_map.astype('uint8'))
# plt.show()